<a href="https://colab.research.google.com/github/Ehtisham1053/Optimization-ML-Algorithms/blob/main/Batch_gradient_descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will learn about the batch gradient descent and then apply this to find the values of m and b in multiple linear regression

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv('audi.csv')
x = df.drop('price', axis=1)
y = df['price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
c = ColumnTransformer([('encoder', OneHotEncoder(handle_unknown='ignore' , sparse_output=False , drop='first'), ['transmission', 'fuelType', 'model']),
                       ('scaler', StandardScaler(), ['mileage', 'tax', 'mpg', 'engineSize', 'year'])

                       ], remainder='passthrough')

x_train = c.fit_transform(x_train)
x_test = c.transform(x_test)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


# Batch Gradient descent Class

In [3]:
import numpy as np

class BatchGradientDescent:
    def __init__(self, learning_rate=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.m = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.m = np.zeros(n_features)
        self.b = 0

        for _ in range(self.epochs):
            y_pred = np.dot(X, self.m) + self.b
            error = y_pred - y

            # Compute gradients
            dm = (1/n_samples) * np.dot(X.T, error)
            db = (1/n_samples) * np.sum(error)

            # Update parameters
            self.m -= self.learning_rate * dm
            self.b -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.m) + self.b

    def mse(self, X, y):
        return np.mean((self.predict(X) - y) ** 2)

    def mae(self, X, y):
        return np.mean(np.abs(self.predict(X) - y))

    def rmse(self, X, y):
        return np.sqrt(self.mse(X, y))

    def r2_score(self, X, y):
        y_mean = np.mean(y)
        ss_total = np.sum((y - y_mean) ** 2)
        ss_residual = np.sum((y - self.predict(X)) ** 2)
        return 1 - (ss_residual / ss_total)

    def adjusted_r2(self, X, y):
        n, k = X.shape
        r2 = self.r2_score(X, y)
        return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


In [4]:
model = BatchGradientDescent(learning_rate=0.1, epochs=1000)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print("MSE:", model.mse(x_test, y_test))
print("MAE:", model.mae(x_test, y_test))
print("RMSE:", model.rmse(x_test, y_test))
print("R² Score:", model.r2_score(x_test, y_test))
print("Adjusted R² Score:", model.adjusted_r2(x_test, y_test))


MSE: 24438367.74617116
MAE: 3033.4305302614703
RMSE: 4943.517750162445
R² Score: 0.8382951845367729
Adjusted R² Score: 0.8357541088652078


#Batch Gradient Descent (BGD)
Batch Gradient Descent computes the gradients using the entire dataset before updating the model parameters. This means each update is based on the sum of all the errors across all training samples. It ensures a smooth and stable convergence since it moves in the direction of the global minimum without much variance.

##When to Use:
BGD is ideal when the dataset is small to moderately sized, as it requires loading the entire dataset into memory at once. It works well when computational resources are sufficient and when the loss function has a smooth, convex shape.

##Advantages:
* Provides a stable convergence path and is less noisy.
* Guarantees reaching the global minimum for convex functions.
* Efficient when working with small datasets.

##Disadvantages:
* Computationally expensive for large datasets as it requires the entire dataset for each update.
* Can be slow to converge, especially for high-dimensional data.
* Not suitable for streaming data, as it needs all data at once.

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

bgd_model = LinearRegression()
bgd_model.fit(x_train, y_train)
y_pred_bgd = bgd_model.predict(x_test)
mse_bgd = mean_squared_error(y_test, y_pred_bgd)
r2_bgd = r2_score(y_test, y_pred_bgd)

print("Batch Gradient Descent Results:")
print(f"Mean Squared Error: {mse_bgd}")
print(f"R² Score: {r2_bgd}")


Batch Gradient Descent Results:
Mean Squared Error: 15861863.853997568
R² Score: 0.8950445547732881
